#Введение

В данном ноутбуке решается следующая задача: в наличии имеется выборка отзывов, собранных из сети. Выборка достаточно большая (около 17000 записей), что дает возможность отфильтровать ее, руководствуясь сходством с отзывами из тестовой выборки. Для определения сходства двух отзывов выбран алгоритм генерации векторных представлений (embeddings) каждого из сравниваемых отзывов с последующим расчетом косинусного расстояния между полученными векторными представлениями.

#Установка и импорт необходимых библиотек

In [ ]:
!pip install datasets transformers[sentencepiece]

In [3]:
import pandas as pd
import numpy as np
import gc
import os
import random
import pickle
import pickle
import requests
import gc

from sklearn.metrics.pairwise import cosine_similarity

from bs4 import BeautifulSoup
from tqdm.auto import tqdm

import torch
from torch import nn
from torch.utils.data import DataLoader, Subset

from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, AutoModel
from transformers import AdamW
from transformers import get_scheduler

from datasets import Dataset, load_dataset
from datasets.dataset_dict import DatasetDict

#Загрузка данных

In [4]:
train_url = 'https://raw.githubusercontent.com/chekhovana/courses/main/ml_stepik/' + \
    '6_final_project/week6_kaggle/data/reviews.csv'

def load_train_dataset():
    df = pd.read_csv(train_url, sep='\t')
    train_dataset = Dataset.from_pandas(df)
    return train_dataset

In [5]:
def load_test_dataset():
    url = 'https://raw.githubusercontent.com/chekhovana/courses/main/ml_stepik/' + \
        '6_final_project/week6_kaggle/data/test.csv'

    content = requests.get(url).content
    bs = BeautifulSoup(content)
    reviews = [r.text for r in bs.findAll('review')]
    test_dataset = Dataset.from_dict({'review': reviews})
    return test_dataset

#Генерация векторных представлений

In [6]:
def seed_all():
    seed = 42
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def get_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    return torch.device('cpu')

device = get_device()
device

device(type='cuda')

In [7]:
def tokenize_dataset(dataset, tokenizer):
    features = list(dataset.features.keys())
    def tokenize_function(example):
        return tokenizer(example["review"], truncation=True, max_length=512)

    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    tokenized_dataset = tokenized_dataset.remove_columns(features)
    tokenized_dataset.set_format("torch")
    return tokenized_dataset

In [8]:
def get_embeddings(model, tokenizer, dataset):
    model.eval()
    embeddings = []
    tokenized_dataset = tokenize_dataset(dataset, tokenizer)
    loader = DataLoader(tokenized_dataset)
    for batch in tqdm(loader):
        with torch.no_grad():
            outputs = model(**batch)
        embeddings.append(outputs.pooler_output)
        gc.collect()
        torch.cuda.empty_cache()

    return torch.cat(embeddings).cpu().numpy()

seed_all()
checkpoint = 'DeepPavlov/rubert-base-cased-sentence'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModel.from_pretrained(checkpoint)
train_dataset = load_train_dataset()
test_dataset = load_test_dataset()
print(len(train_dataset), len(test_dataset))
train_embeddings = get_embeddings(model, tokenizer, train_dataset)
test_embeddings = get_embeddings(model, tokenizer, test_dataset)

17275 100


  0%|          | 0/18 [00:00<?, ?ba/s]

  0%|          | 0/17275 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Сохраняем полученные представления в файлы для последующего использования

In [13]:
with open('embeddings_train.pickle', 'wb') as handle:
    pickle.dump(train_embeddings, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('embeddings_test.pickle', 'wb') as handle:
    pickle.dump(test_embeddings, handle, protocol=pickle.HIGHEST_PROTOCOL)

При необходимости можно загрузить представления из файлов

In [14]:
with open('embeddings_train.pickle', 'rb') as handle:
    train_embeddings = pickle.load(handle)
with open('embeddings_test.pickle', 'rb') as handle:
    test_embeddings = pickle.load(handle)
train_embeddings.shape, test_embeddings.shape

((17275, 768), (100, 768))

#Фильтрация обучающей выборки

Рассчитываем попарные косинусные расстояния между отзывами обучающей и тестовой выборок. Затем используем эти расстояния для фильтрации обучающей выборки. Для получения сбалансированной выборки положительные и отрицательные отзывы фильтруем независимо.

In [11]:
similarity = cosine_similarity(test_embeddings, train_embeddings)

def sort_by_similarity(x, n, indexes):
    x = x.T.reshape(-1, )[::-1]
    res = []
    for i in x:
        if i not in res and i in indexes:
            res.append(i)
        if len(res) == n:
            break
    return res

args = np.argsort(similarity)
df = pd.read_csv(train_url, sep='\t')
positive_indexes = df[df['label'] == 1].index.values.tolist()
negative_indexes = df[df['label'] == 0].index.values.tolist()
n_samples = 1000
similar_positive_indexes = sort_by_similarity(args, n_samples, positive_indexes)
similar_negative_indexes = sort_by_similarity(args, n_samples, negative_indexes)
similar_indexes = similar_negative_indexes + similar_positive_indexes

Перемешиваем отфильтрованную обучающую выборку и сохраняем ее в файл

In [12]:
df_filtered = df.iloc[similar_indexes].sample(frac=1)
df_filtered.to_csv(f'reviews_filtered_{len(similar_indexes)}.csv', sep='\t', 
                   index=False)